In [3]:
import cv2
import numpy as np
import glob

# 设置寻找亚像素角点的参数，采用的停止准则是最大循环次数30和最大误差容限0.001
criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 30, 0.001)

# 获取标定板角点的位置
objp = np.zeros((8 * 8, 3), np.float32)             # (x, y, 0)
objp[:, :2] = np.mgrid[0:8, 0:8].T.reshape(-1, 2)   # 将世界坐标系建在标定板上，所有点的Z坐标全部为0，所以只需要赋值x和y

obj_points = []  # 存储3D点
img_points = []  # 存储2D点

# 获取对应文件夹下的所有图片，进行标定工作
images = glob.glob("images/scaled/circle/*.jpg")
# 需要对图片进行排序，不然之后的绘制过程可能会因为乱序而没有效果
images.sort()

# 遍历所有图片，寻找角点，并记录，等待后续标定使用
i=0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    size = gray.shape[::-1]
    ret, corners = cv2.findChessboardCorners(gray, (8, 8), None)
    
    if ret:
        obj_points.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)  # 在原角点的基础上寻找亚像素角点
        if [corners2]:
            img_points.append(corners2)
        else:
            img_points.append(corners)

        cv2.drawChessboardCorners(img, (8, 8), corners, ret)  # 记住，OpenCV的绘制函数一般无返回值
        i+=1
        cv2.imwrite('conimg'+str(i)+'.jpg', img)
        cv2.waitKey(1500)

# 显示可用图片的数量
print(len(img_points))
cv2.destroyAllWindows()

# 标定步骤
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, size, None, None)

print("ret:", ret)
print("mtx:\n", mtx)        # 内参数矩阵
print("dist:\n", dist)      # 畸变系数 distortion cofficients = (k_1, k_2, p_1, p_2, k_3)
print("rvecs:\n", rvecs)    # 旋转向量, 外参数
print("tvecs:\n", tvecs )   # 平移向量, 外参数

print("-----------------------------------------------------")
img = cv2.imread(images[2])
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h)) # 显示更大范围的图片（正常重映射之后会删掉一部分图像）
print(newcameramtx)

# 消除畸变，并显示效果
print("------------------使用undistort函数-------------------")
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
x,y,w,h = roi
dst1 = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult3.jpg', dst1)
print ("方法一:dst的大小为:", dst1.shape)

True

## change the world coordinate to camera coordinate
$$
\begin{bmatrix}
x\\y\\z
\end{bmatrix}_{camera}=R
\begin{bmatrix}
X\\Y\\Z
\end{bmatrix}_{world}+t
$$

In [6]:
# 重建得到相机的位置
result = []

for i in range(len(rvecs)):
    rot = cv2.Rodrigues(rvecs[i])
    R = rot[0]
    # 参见公式
    result.append(np.dot(np.linalg.inv(R), -tvecs[i]))

In [7]:
# 创建ply文件, 需要安装，pip install plyfile
from plyfile import PlyData, PlyElement

def write_ply(save_path, points, pts, text=True):
    # points[9, :]: the camera position in world coordinate
    points = [(points[i, 0], points[i, 1], points[i, 2]) for i in range(points.shape[0])]
    # objp[64, :]: the regular chessboard position in world coordinate
    for i in range(pts.shape[0]):
        points.append((pts[i, 0], pts[i, 1], pts[i, 2]))
    # change the points list into np.array
    vertex = np.array(points, dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])
    el = PlyElement.describe(vertex, 'vertex', comments=['vertices'])
    PlyData([el], text=text).write(save_path)

write_ply('new.ply', np.array(result), objp, True)

In [8]:
# 给定图片上的若干点，我们将其连接起来形成立方体的样子
def draw_lines(img, points):
    pairs = []
    pairs += [(i, i+4) for i in range(4)]           # vertical lines
    pairs += [(i, (i+1)%4) for i in range(4)]       # bottom line
    pairs += [(i+4, (i+1)%4 + 4) for i in range(4)] # top line
    for tup in pairs:
        pt1 = (int(points[tup[0], 0]), int(points[tup[0], 1]))
        pt2 = (int(points[tup[1], 0]), int(points[tup[1], 1]))
        cv2.line(img, pt1, pt2, (0, 0, 255), 4, 3)

## change the world coordinate to image coordinate
$$
sm' = A[R|t]M'\\

s\begin{bmatrix}
u\\v\\1
\end{bmatrix}=
\begin{bmatrix}
f_x &0   &c_x\\
0   &f_y &c_y\\
0   &0   &1
\end{bmatrix}
\begin{bmatrix}
r_{11} &r_{12} &r_{13} &t_1\\
r_{21} &r_{22} &r_{23} &t_2\\
r_{31} &r_{32} &r_{33} &t_3
\end{bmatrix}
\begin{bmatrix}
X\\Y\\Z\\1
\end{bmatrix}
$$
+ $(X,Y,Z)$ are the coordinates of a 3D point in the world coordinate
+ $(u,v)$ are the coordiantes of the projection point in pixels
+ $A$ is a camera matrix, or a matrix of intrinsic parameters
+ $(c_x, c_y)$ is a principal point that is usually at the image center
+ $f_x,f_y$ are the focal lengths expressed in pixel units

In [9]:
# 图片数量
num_of_pic = 9

# origin中储存了立方体八个顶点在世界坐标系下的坐标
origin = [[2, 2, 0], [2, 3, 0], [3, 3, 0], [3, 2, 0], [2, 2, -1], [2, 3, -1], [3, 3, -1], [3, 2, -1]]
origin = np.array(origin)
origin = np.hstack((origin, np.ones(8).reshape(8, 1)))

# 分别对每张图片进行重建
for i in range(num_of_pic):
    # 重建第一组点
    rot = cv2.Rodrigues(rvecs[i])
    R = rot[0]
    # hstack to get [R T]，3 x 4
    mat = np.hstack((R, tvecs[i].reshape((3, 1))))
    # 利用公式算出世界坐标系下对应点在图像坐标系下的坐标
    tmp1 = np.dot(mat, origin.T)
    out = np.dot(mtx, tmp1)
    out = out / out[2, :]
    # open the object image
    filename = './images/scaled/circle/' + str(i+1) + '.jpg'
    img = cv2.imread(filename)
    # draw cubic onto the object image
    draw_lines(img, out.T)
    # save the adjusted image
    cv2.imwrite(str(i+1)+'.png', img)